In [1]:
import pysam
import os
import sys
from sys import getsizeof
import time
import pandas as pd
import polars as pl
from collections import defaultdict

from collections import defaultdict
import pandas as pd
#from matplotlib import pyplot as plt
import numpy as np
import time
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm


from glob import glob

directory_path = os.path.abspath(os.path.join('../src/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from read_process import incorporate_replaced_pos_info,incorporate_insertions_and_deletions,\
get_positions_from_md_tag,reverse_complement,get_edit_information,get_edit_information_wrapper,\
has_edits,get_total_coverage_for_contig_at_position,\
print_read_info, update_coverage_array, get_read_information, get_hamming_distance, remove_softclipped_bases,find

from utils import get_contigs_that_need_bams_written, get_contig_lengths_dict, get_intervals, index_bam, write_rows_to_info_file, write_header_to_edit_info, \
write_read_to_bam_file, remove_file_if_exists, make_folder, concat_and_write_bams_wrapper, make_edit_finding_jobs

from core import run_edit_identifier, run_bam_reconfiguration

import os, psutil


# Preload which barcodes to use...

In [2]:
barcodes_list_path = '/projects/ps-yeolab3/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/cellranger/results/ms_hippo_stamp_EIF4A_batch2/outs/filtered_feature_bc_matrix/barcodes.tsv.gz'

In [3]:
barcode_whitelist = set(pd.read_csv(barcodes_list_path, names=['barcodes']).barcodes.tolist())

In [4]:
len(barcode_whitelist)

39200

# ~~~~~~~~~~~~~~~~~~
# Multi-processing enabled
# ~~~~~~~~~~~~~~~~~~

# An example on a full 10x bam

#### in 10X's bam file, xf=25 means that read is uniquely mapped to a genome, and was used for counting UMI. So we should only look at reads with xf=25 from the 10X bam.

In [5]:
#bampath = '/projects/ps-yeolab5/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_batch2/filtered_possorted_ms_hippo_stamp_bam/filtered_keep_xf25_possorted_genome_with_header.bam_MD.bam'
bampath = '/projects/ps-yeolab3/ekofman/sailor2/data/groups_0_1_2_3_4_5_6_7_8_9_10_11_merged.bam'

samfile = pysam.AlignmentFile(bampath, "rb")
samfile_header = str(samfile.header)

output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset'
#output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig'

# Print info?
verbose = False 


# How many subcontigs to split each contig into to leverage multi-processing
num_intervals_per_contig = 16

# Go through every read and identify all edits

In [6]:


overall_label_to_list_of_contents, results, overall_time, overall_total_reads, total_seconds_for_reads = run_edit_identifier(
    bampath, 
    output_folder, 
    barcode_whitelist,
    contigs=['1', '2'],
    num_intervals_per_contig=num_intervals_per_contig, 
    verbose=verbose)


	Contig 1
	Contig 2
32 total jobs


100%|████████████████████████████████████████████████████████████████████████████████| 32/32 [00:07<00:00,  4.39it/s]


In [7]:
print("Total time: {} seconds".format(overall_time))
print("Total time: {} minutes".format(overall_time/60))

Total time: 7.399926118552685 seconds
Total time: 0.12333210197587809 minutes


Memory: 230 Gigabytes

# More helper functions

# Combine all of the reads (string representation) for each barcode
## Groups the results from each sub-contig segment above, for example the reads from the first half of chr1 and those from the second half.

### which finished?

In [8]:
# Make a subfolder into which the split bams will be placed
split_bams_folder = '{}/split_bams'.format(output_folder)
make_folder(split_bams_folder)
    

In [9]:
from glob import glob

contigs_to_generate_bams_for = get_contigs_that_need_bams_written(overall_label_to_list_of_contents, split_bams_folder)

In [10]:
contigs_to_generate_bams_for

[]

### Generate list of jobs to be multiprocessed

In [11]:
from collections import OrderedDict
import time



In [12]:
total_bam_generation_time = run_bam_reconfiguration(split_bams_folder, bampath, overall_label_to_list_of_contents, contigs_to_generate_bams_for)

0it [00:00, ?it/s]


In [14]:
print("Total time to concat and write bams: {} minutes".format(round(total_bam_generation_time/60)))

Total time to concat and write bams: 0 minutes


# Second loop to get coverage at sites with edits

In [ ]:
from glob import glob
import os
import sys
from collections import defaultdict
import pandas as pd
import polars as pl

directory_path = os.path.abspath(os.path.join('../src/'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
    
from utils import get_edit_info_for_barcode_in_contig_wrapper

#output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig-subset'
output_folder = '/projects/ps-yeolab3/ekofman/sailor2/scripts/full_test-highmem_bccontig'


splits = [i.split("/")[-1].split('_edit')[0] for i in sorted(glob('{}/edit_info/*'.format(output_folder)))]
print("Accessing split bams: {}".format(', '.join(sorted(splits))))

### Gather the edit information generated for each subcontig, and group by contig so we only have 1 edit information dataframe to process per contig

In [ ]:
all_edit_info_for_barcodes = []

edit_info_grouped_per_contig = defaultdict(lambda:[])
edit_info_grouped_per_contig_combined = defaultdict(lambda:[])

num_splits = len(splits)
print("Grouping edit information outputs by contig...")
for i, split in enumerate(splits):
    if i%10 == 0:
        print("\t{}/{}...".format(i, num_splits))
        
    contig = split.split("_")[0]
    print("contig: {}, {}".format(contig, split))
    
    barcode_to_coverage_dict = defaultdict()    
    
    barcode_to_coverage_dict = defaultdict()
    edit_info_file = '{}/edit_info/{}_edit_info.tsv'.format(output_folder, split)
    edit_info_df = pd.read_csv(edit_info_file, sep='\t')
    edit_info_df['position'] = edit_info_df['position'].astype(int)
    edit_info_df['base_quality'] = edit_info_df['base_quality'].astype(int)
    edit_info_df['mapping_quality'] = edit_info_df['mapping_quality'].astype(int)
    edit_info_df['dist_from_end'] = edit_info_df['dist_from_end'].astype(int)

    edit_info = pl.from_pandas(edit_info_df) 
    
    for n in ["A", "C", "G", "T"]:
        suffix = '{}-1'.format(n)
        print("\tsuffix: {}".format(suffix))
        edit_info_subset = edit_info.filter(pl.col("barcode").str.ends_with(suffix))
        
        edit_info_grouped_per_contig["{}_{}".format(contig, n)].append(edit_info_subset)
    
    del edit_info_df
    
print("Done grouping! Concatenating ...")



In [ ]:
glob(edit_info_file)

In [ ]:
for contig, list_of_edit_info_dfs in edit_info_grouped_per_contig.items():
    edit_info_grouped_per_contig_combined[contig] = pl.concat(list_of_edit_info_dfs)

print("Done concatenating!")

### Get coverage at edit positions for each contig

##### Merge across contigs for each barcode???

In [ ]:
import pandas as pd

pd.options.mode.chained_assignment = None 


def get_job_params_for_coverage_for_edits_in_contig(edit_info_grouped_per_contig_combined, output_folder):
    job_params = []
    
    for contig, edit_info in edit_info_grouped_per_contig_combined.items():
        print(contig)
        print('Num edits pre filter: {}'.format(len(edit_info)))
        edit_info = edit_info.filter(pl.col("base_quality") > 15)
        print('\tNum edits post filter: {}'.format(len(edit_info)))
        
        #unique_barcodes = list(edit_info.unique("barcode")["barcode"])
        #unique_contigs = list(edit_info.unique("contig")["contig"])
        #print('Num unique contigs: {}'.format(len(unique_contigs)))
        
        job_params.append([edit_info, contig, output_folder])  
    return job_params
    
coverage_counting_job_params = get_job_params_for_coverage_for_edits_in_contig(edit_info_grouped_per_contig_combined, 
                                                                output_folder)
len(coverage_counting_job_params)

### This is going at rate of 1 items per seconds... which would take several hours...

In [ ]:
#edit_info_plus_coverage_df = get_edit_info_for_barcode_in_contig_wrapper(coverage_counting_job_params[0])

In [ ]:
len(coverage_counting_job_params)

In [ ]:
from multiprocessing import get_context
import time
from multiprocessing import Pool
import multiprocessing
from tqdm import tqdm
import numpy as np

start_time = time.perf_counter()

results = []
# Spawn has to be used instead of the default fork when using the polars library
with get_context("spawn").Pool(processes=16) as p:
    max_ = len(coverage_counting_job_params)
    with tqdm(total=max_) as pbar:
        for _ in p.imap_unordered(get_edit_info_for_barcode_in_contig_wrapper, coverage_counting_job_params):
            pbar.update()
            results.append(_)
            
total_time = time.perf_counter() - start_time

In [ ]:
print(total_time/60)

In [ ]:
len(results)

In [ ]:
all_edit_info = pd.concat(results)

In [ ]:
all_edit_info.groupby(['ref', 'alt']).count()


In [ ]:
total_time/60

In [ ]:
all_edit_info.to_csv('{}/all_edit_info.tsv'.format(output_folder), sep='\t')

# Group by site to get final total edit and coverage counts at each site

In [ ]:
all_edit_info.head()


# Verify C>T ratios

In [ ]:
all_edit_info.groupby(['ref', 'alt']).count()


In [ ]:
all_edit_info.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
plt.title("All edits")

base_quality_thresh = 15
all_edit_info[all_edit_info.base_quality > base_quality_thresh].groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
plt.title("Edits with base quality > {}".format(base_quality_thresh))

all_edit_info_filtered = all_edit_info[all_edit_info.base_quality > base_quality_thresh]


In [ ]:
example_new_ct =  all_edit_info_filtered[(all_edit_info_filtered.ref == 'C') & (all_edit_info_filtered.alt == 'T')].sort_values('position')

In [ ]:
len(example_new_ct)

In [ ]:
example_new_ct

# Cells that do have STAMP expressed versus don't...?

In [ ]:
stamp_expression_path = \
'/projects/ps-yeolab3/ekofman/Sammi/MouseBrainEF1A_SingleCell_EPR_combined/\
4.1_cells_with_middling_stamp/stamp_expression_for_all_used_cells.tsv'

In [ ]:
stamp_expression_df = pd.read_csv(stamp_expression_path, sep='\t', index_col=0)

In [ ]:
stamp_expression_df.Stamp.hist(bins=50)

In [ ]:
all_edit_info_filtered['edit'] = all_edit_info_filtered['ref'] + '>' + all_edit_info_filtered['alt']

In [ ]:
fractions_ct = []
threshs = [0, 0.5, 1, 2, 3, 4, 5, 6, 6.5, 6.6]
for thresh in threshs:
    print(thresh)
    barcodes_at_stamp_thresh = stamp_expression_df[stamp_expression_df.Stamp > thresh].index
    
    all_edit_info_filtered_in_stamp_level = all_edit_info_filtered[
        all_edit_info_filtered.barcode.isin(barcodes_at_stamp_thresh)]
    
    all_edit_info_filtered_in_stamp_level.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
    plt.title("Edit Type Distribution for Cells with STAMP expression above {}".format(thresh))
    
    fraction_ct = len(all_edit_info_filtered_in_stamp_level[all_edit_info_filtered_in_stamp_level['edit'] == 'C>T'])/len(all_edit_info_filtered_in_stamp_level)
    fractions_ct.append(fraction_ct)
    
    

In [ ]:
plt.plot(threshs, fractions_ct)
plt.ylabel("Fraction of total edits that are C>T")
plt.xlabel("STAMP expression minimum")
plt.title("Enrichment for C>T edits within cells filtered by STAMP threshold")

In [ ]:
fractions_ct_low = []
threshs = [1, 2, 3, 4, 5, 6, 6.5, 6.6]
for thresh in threshs:
    print(thresh)
    barcodes_at_stamp_thresh = stamp_expression_df[stamp_expression_df.Stamp < thresh].index
    
    all_edit_info_filtered_in_stamp_level = all_edit_info_filtered[
        all_edit_info_filtered.barcode.isin(barcodes_at_stamp_thresh)]
    
    all_edit_info_filtered_in_stamp_level.groupby(['ref', 'alt']).count().plot(kind='barh', legend=False)
    plt.title("Edit Type Distribution for Cells with STAMP expression below {}".format(thresh))
    
    fraction_ct = len(all_edit_info_filtered_in_stamp_level[all_edit_info_filtered_in_stamp_level['edit'] == 'C>T'])/len(all_edit_info_filtered_in_stamp_level)
    fractions_ct_low.append(fraction_ct)

In [ ]:
plt.plot(threshs, fractions_ct_low)
plt.ylabel("Fraction of total edits that are C>T")
plt.xlabel("STAMP expression maximum")
plt.title("Enrichment for C>T edits within cells filtered by STAMP threshold")